In [1]:
import pandas as pd
from bert_score import BERTScorer

from src.config import mturk_simjudgement_file, mturk_substitution_rating_scores

import warnings
warnings.filterwarnings('ignore')

In [2]:
import time
start = time.time()
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
end = time.time()
print(end - start)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


13.971173524856567


In [3]:
def get_scores(single_cands, multi_refs):
    P_mul, R_mul, F_mul = scorer.score([single_cands], [multi_refs])
    return P_mul.item(), R_mul.item(), F_mul.item()

def scores(line):
    return get_scores(line['rec_item'].lower(), line['gt_item'].lower())

In [4]:
import time
start = time.time()
df = pd.read_csv(mturk_simjudgement_file)
df['P'], df['R'], df['F1'] = zip(*df.apply(scores, axis=1))
end = time.time()
print(end - start)

908.3813455104828


In [5]:
cols = ['gt_item', 'rec_item', 'gt_item_id', 'rec_item_id', 'hint', 'SimJudgement', 'P', 'R', 'F1']
df = df[cols]
# df = df[cols].rename(columns={'R':'BERTScore', 'F1':'BERTScore-F1'})
df.to_csv(mturk_substitution_rating_scores['BERTScores'], index=False)

In [6]:
df.corr().iloc[3:4, 4:]

,P,R,F1
SimJudgement,0.33856,0.343378,0.375478
